In [1]:
#importing the libraries we need
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torchvision
import cv2
from torch.utils.data import Dataset, DataLoader 
from torch.autograd import Variable
from torch import nn 

In [2]:
pre_model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11_bn', pretrained=True)

Using cache found in C:\Users\Mohie Eldeen Elzeiny/.cache\torch\hub\pytorch_vision_v0.6.0


In [3]:
torch.save(pre_model.state_dict(), 'Saved weights/vgg11_bn.pth')

In [4]:
print(pre_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [36]:
class I3D (nn.Module):
    
    def __init__(self , output_features = 101):
        
        super().__init__()
        
        self.output_features = output_features
        
        self.features = nn.Sequential(
            nn.Conv3d(3, 64, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(64 ),
            nn.ReLU(inplace=True),
            nn.AvgPool3d((1,2,2)),
            nn.Conv3d(64, 128, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(128 ),
            nn.ReLU(inplace=True),
            nn.AvgPool3d(2),
            nn.Conv3d(128, 256, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(256 ),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 256, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(256 ),
            nn.ReLU(inplace=True),
            nn.AvgPool3d(2),
            nn.Conv3d(256, 512, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(512 ),
            nn.ReLU(inplace=True),
            nn.Conv3d(512, 512, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(512 ),
            nn.ReLU(inplace=True),
            nn.AvgPool3d(2),
            nn.Conv3d(512, 512, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(512 ),
            nn.ReLU(inplace=True),
            nn.Conv3d(512, 512, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(512 ),
            nn.ReLU(inplace=True),
            nn.AvgPool3d(2)    
        )
        
        
        self.features_flow = nn.Sequential(
            nn.Conv3d(3, 64, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(64 ),
            nn.ReLU(inplace=True),
            nn.AvgPool3d((1,2,2)),
            nn.Conv3d(64, 128, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(128 ),
            nn.ReLU(inplace=True),
            nn.AvgPool3d(2),
            nn.Conv3d(128, 256, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(256 ),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 256, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(256 ),
            nn.ReLU(inplace=True),
            nn.AvgPool3d(2),
            nn.Conv3d(256, 512, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(512 ),
            nn.ReLU(inplace=True),
            nn.Conv3d(512, 512, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(512 ),
            nn.ReLU(inplace=True),
            nn.AvgPool3d(2),
            nn.Conv3d(512, 512, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(512 ),
            nn.ReLU(inplace=True),
            nn.Conv3d(512, 512, kernel_size=(3, 3 , 3), padding=1),
            nn.BatchNorm3d(512 ),
            nn.ReLU(inplace=True),
            nn.AvgPool3d(2)    
        )
        
        
        self.classification = nn.Sequential(
            
            nn.Linear(in_features=16384, out_features=2048),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=2048, out_features=512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=512, out_features=self.output_features),
            nn.LogSoftmax(dim = 1)
        )
        
        
    def forward(self , x , y):
        
        #expected input shape of x and y is (batch , 3 , 16 , 128 , 128)
        #x is the frames and y is the optical flow of the frames
        
        x = self.features(x)
        
        y = self.features_flow(y)
        
        
        
        x = x.flatten(1)
        
        y = y.flatten(1)
        
        
        out = torch.cat( [x , y] , dim = 1 )
        
        out = self.classification(out)
        
        
        
        return (out)
        
        

In [60]:
mo = I3D(101)

In [41]:
print(mo)

I3D(
  (features): Sequential(
    (0): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): AvgPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0)
    (4): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (5): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): AvgPool3d(kernel_size=2, stride=2, padding=0)
    (8): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (9): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (12): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): AvgPool3d(kernel_s

In [59]:
def cloneParams (target_model , source_model):
    
    """
    clones the parameters of a pre-trained model to another new model.
    the function is mainly made to clone the parameters of 2D CNN model into another 3D CNN model that has the same architecture
    
    =====================
    Args :
    =====================
    
    target_model : the model that's targeted to update its weights (the 3D model)
    
    source_model : the pretrained model that we are going to clone its parameter
    """

    for layer in target_model.state_dict().keys():
        
        if layer.replace('_flow' , '') in source_model.state_dict().keys():

            if target_model.state_dict()[layer].shape == source_model.state_dict()[layer.replace('_flow' , '')].shape :

                target_model.state_dict()[layer].data.copy_( source_model.state_dict()[layer.replace('_flow' , '')] )

            else :

                temp = target_model.state_dict()[layer].shape[2] #the temporal dimension

                for i in range (temp) :

                    target_model.state_dict()[layer][:, :, i, :, :].data.copy_(source_model.state_dict()[layer.replace('_flow' , '')] / temp)
                    
    return target_model
            
            
        
        

In [64]:
mo = cloneParams (mo , pre_model)

In [80]:
torch.save(mo.state_dict(), 'Saved weights/I3D_vgg11.pth')

In [4]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11_bn', pretrained=True)

Using cache found in C:\Users\Mohie Eldeen Elzeiny/.cache\torch\hub\pytorch_vision_v0.6.0


In [11]:
from torch import nn

In [ ]:
features = nn.Sequential([
    0 =  Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    1 =  BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (17): ReLU(inplace=True)
    (18): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (19): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (20): ReLU(inplace=True)
    (21): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (22): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (23): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (24): ReLU(inplace=True)
    (25): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (26): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (27): ReLU(inplace=True)
    (28): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    ])
  (avgpool): AdaptiveAvgPool2d(output_size=(7, 7))
  (classifier): Sequential(
    (0): Linear(in_features=25088, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=4096, out_features=1000, bias=True)
  )

In [5]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [ ]:
#that's the code to get the progress numbers again...just uncomment and use it

with open("CNN Acc and Loss.txt", "r") as file:
    nums = file.read()
    
tr_loss_lis, tr_acc_lis, val_loss_lis, val_acc_lis = nums.split('seperator')[0:4]
tr_loss_lis = [float(re.findall(r'\d+\.\d+' , num)[0]) for num in tr_loss_lis.split(',')]
tr_acc_lis = [float(re.findall(r'\d+\.\d+' , num)[0]) for num in tr_acc_lis.split(',')]
val_loss_lis = [float(re.findall(r'\d+\.\d+' , num)[0]) for num in val_loss_lis.split(',')]
val_acc_lis = [float(re.findall(r'\d+\.\d+' , num)[0]) for num in val_acc_lis.split(',')]

import time
epoch = 0
s = 0
for i in range (len(tr_loss_lis)):
    
    time.sleep(0.0005)
    
    if s==0 :
        
        print(f'[{epoch+1}, {s}] train_loss:{tr_loss_lis[i] : .5f} train_acc:{100 * tr_acc_lis[i] : 0.5f} val_loss:{val_loss_lis[i] : 0.5f} val_acc:{100 * val_acc_lis[i] : 0.5f}' )
        s = 10
    else:
    
        print(f'[{epoch+1}, {s}] train_loss:{tr_loss_lis[i] / 10 : .5f} train_acc:{100 * tr_acc_lis[i] / 10 : 0.5f} val_loss:{val_loss_lis[i] / 10 : 0.5f} val_acc:{100 * val_acc_lis[i] / 10 : 0.5f}' )
        s+=10
        
        if s == 9970:
            s = 0

In [1]:
import pandas as pd

In [4]:
torch.float(torch.Tensor(15.2))

TypeError: new(): data must be a sequence (got float)

In [2]:
test_videos = pd.read_csv('video_test_1_classes.csv')
train_videos = pd.read_csv('video_train_1.csv')

In [3]:
len(test_videos)

3783

In [54]:
def trainValSpliteFromVideoNames(vali_ratio , train_videos):
    """
    this function is used to split the videos into validation and train data by spliting the video names in random
    (the output is shuffled)

    Args :
        vali_ratio (float) : the ratio by which the data will be split
        train_videos (pandas dataFrame) : the data frame that holds the names and the classes of the videos

    Returns :

        tr_frames : dataframe that holds the new train frames
        val_frames : dataframe that holds the new validation frames

    """

    videos_idx = [i for i in range(len(train_videos))] #creating a list to hold the indeces of the train data
    np.random.shuffle (videos_idx) #shuffling the indeces

    train_num = int( len(train_videos) * (1-vali_ratio)  ) #the size of train data

    tr_videos = train_videos.iloc[  videos_idx[ : train_num]   , :] #assigning the train data by the threshold defined
    val_videos = train_videos.iloc[  videos_idx[train_num : ]   , :] #assigning the validation data by the threshold defined

    tr_videos = tr_videos.reset_index().drop(['index'] , axis= 1) #reseting the indeces to avoid some problems
    val_videos = val_videos.reset_index().drop(['index'] , axis= 1) #reseting the indeces to avoid some problems
    
    #in this for loop we will make a dataFrame that holds the names of the frames and thier responding classes for the train data
    tr_frames = []

    for i in range (len(tr_videos)):

        vid_name = tr_videos['name'][i]
        vid_class = tr_videos['class'][i]
        vid_frames = tr_videos['num_frames'][i]

        tr_frames += [(vid_name + '_' + str(n) + '.jpg' , vid_class) for n in range(1,vid_frames+1)]

    tr_frames = pd.DataFrame(tr_frames , columns= ['name' , 'class'])
    
    
    #in this for loop we will make a dataFrame that holds the names of the frames and thier responding classes for the validation data
    val_frames = []

    for i in range (len(val_videos)):

        vid_name = val_videos['name'][i]
        vid_class = val_videos['class'][i]
        vid_frames = val_videos['num_frames'][i]

        val_frames += [(vid_name + '_' + str(n) + '.jpg' , vid_class) for n in range(1,vid_frames+1)]

    val_frames = pd.DataFrame(val_frames , columns= ['name' , 'class'])

    return tr_frames  ,val_frames

In [55]:
tr_vids , val_vids = trainValSpliteFromVideoNames(0.1 , train_videos)

In [57]:
val_vids.head()

,name,class
0,v_BabyCrawling_g15_c01.avi_1.jpg,4
1,v_BabyCrawling_g15_c01.avi_2.jpg,4
2,v_BabyCrawling_g15_c01.avi_3.jpg,4
3,v_BabyCrawling_g15_c01.avi_4.jpg,4
4,v_BabyCrawling_g15_c01.avi_5.jpg,4


In [52]:
tr_frames = []

for i in range (len(tr_vids)):
    
    vid_name = tr_vids['name'][i]
    vid_class = tr_vids['class'][i]
    vid_frames = tr_vids['num_frames'][i]
    
    tr_frames += [(vid_name + '_' + str(n) + '.jpg' , vid_class) for n in range(1,vid_frames+1)]
    
tr_frames = pd.DataFrame(tr_frames , columns= ['name' , 'class'])

In [53]:
tr_frames

,name,class
0,v_HorseRace_g16_c03.avi_1.jpg,41
1,v_HorseRace_g16_c03.avi_2.jpg,41
2,v_HorseRace_g16_c03.avi_3.jpg,41
3,v_HorseRace_g16_c03.avi_4.jpg,41
4,v_HorseRace_g16_c03.avi_5.jpg,41
5,v_HorseRace_g16_c03.avi_6.jpg,41
6,v_HorseRace_g16_c03.avi_7.jpg,41
7,v_HorseRace_g16_c03.avi_8.jpg,41
8,v_HorseRace_g16_c03.avi_9.jpg,41
9,v_HorseRace_g16_c03.avi_10.jpg,41


In [51]:
tr_vids

,name,class,num_frames
0,v_HorseRace_g16_c03.avi,41,56
1,v_Skiing_g11_c04.avi,81,66
2,v_Diving_g21_c05.avi,26,36
3,v_Skijet_g11_c02.avi,82,42
4,v_YoYo_g16_c03.avi,101,33
5,v_CleanAndJerk_g25_c04.avi,21,51
6,v_SumoWrestling_g15_c04.avi,87,48
7,v_SumoWrestling_g18_c03.avi,87,34
8,v_ParallelBars_g14_c03.avi,57,30
9,v_Typing_g16_c01.avi,95,54


In [ ]:
def testAccFrames (model , test_videos , test_dir  , minibatch ,  device , transfrom = None) :
    """
    computes the accuracy of the model on the test videos for the models that only deals with the frames
    
    ------------------------------------------------------
    Args:
    ------------------------------------------------------
    model : the trained model that will be used to compute the accuracy
    
    test_videos : a data frame that has the names of the videos and thier responding number of frames
    
    test_dir : the directory of the frames folder
    
    transfrom : the transforms to be done on the images
    
    minibatch : the number of frames to enter the model each time for prediction
    
    device : the device that the model is going to be trained on (cuda or cpu)
    
    """
    
    total_acc = 0
    predicted = []
    
    for i , vid_name in enumerate (test_videos['name']) :
        
        
        for i  in range( 1 , test_videos['num_frames'][i] + 1):
            
            frame_dir = vid_name + '_' + str(i) + '.jpg'
            
            frame = cv2.imread(test_dir + frame_dir)
            
            if transfrom :
                
                frame = transfrom(frame)
            
            
            
            
        
        
        
        

In [27]:
test_dir = 'Test_Frames_1/'

In [18]:
vid_name = test_videos['name'][0]


In [23]:
for i in range (1, 17 ,5):
    print(i)

1
6
11
16


In [29]:
cv2.imread(test_dir + frame_names[0]).shape

(240, 320, 3)

In [2]:
class OFFModel (nn.Module):

    def __init__ (self , output_features , batch_size , seq_size) :

        super().__init__()
        
        #the constants 
        self.output_features = output_features
        self.batch_size = batch_size
        self.seq_size = seq_size


        # Feature Generation Sub-network
        self.conv1a = nn.Conv2d(3,16,(3,3) , padding = 2)
        self.relu1a = nn.ReLU(inplace = True)
        self.conv1b = nn.Conv2d(16,16,(3,3) , padding = 1)
        self.relu1b = nn.ReLU(inplace = True)
        self.maxpool1 = nn.MaxPool2d((2,2) , padding = 1)

        self.conv2a = nn.Conv2d(16,32,(3,3) )
        self.relu2a = nn.ReLU(inplace = True)
        self.conv2b = nn.Conv2d(32,32,(3,3) , padding = 1)
        self.relu2b = nn.ReLU(inplace = True)
        self.maxpool2 = nn.MaxPool2d((2,2) , padding = 1)

        self.conv3a = nn.Conv2d(32,64,(3,3))
        self.relu3a = nn.ReLU(inplace = True)
        self.conv3b = nn.Conv2d(64,64,(3,3) , padding = 1)
        self.relu3b = nn.ReLU(inplace = True)
        self.conv3c = nn.Conv2d(64,64,(3,3) , padding = 1)
        self.relu3c = nn.ReLU(inplace = True)
        self.maxpool3 = nn.MaxPool2d((2,2) , padding = 1)

        self.conv4a = nn.Conv2d(64,128,(3,3))
        self.relu4a = nn.ReLU(inplace = True)
        self.conv4b = nn.Conv2d(128,128,(3,3) , padding = 1)
        self.relu4b = nn.ReLU(inplace = True)
        self.avgpool4 = nn.AvgPool2d((2,2) , padding = 1)

        self.ffnn5 = nn.Linear(8192, 2048)
        self.relu5 = nn.ReLU(inplace = True)
        self.drop5 = nn.Dropout(p = 0.3)

        self.ffnn6 = nn.Linear(2048 , 101)
        self.softmax = nn.LogSoftmax(dim = 1)
        
        #OFF Sub-network unit one
        self.off1conv1 = nn.Conv2d(16 , 64 , (1,1)) #for dimentionality control
        self.off1relu1 = nn.ReLU(inplace = True)
        
        self.off1conv2 = nn.Conv2d(192 , 64 , (1,1) )
        self.off1relu2 = nn.ReLU(inplace = True)
        
        self.off1conv3 = nn.Conv2d(64 , 64 , (3,3) , padding= 1)
        self.off1relu3 = nn.ReLU(inplace = True)
        self.off1conv4 = nn.Conv2d(64 , 64 , (3,3) , padding= 1)
        self.off1relu4 = nn.ReLU(inplace = True)
        self.off1conv5 = nn.Conv2d(64 , 64 , (3,3) , padding= 1)
        self.off1relu5 = nn.ReLU(inplace = True)
        
        self.off1avgpool = nn.AvgPool2d((2,2), padding = 1)
        
        #OFF Sub-network unit two
        self.off2conv1 = nn.Conv2d(32 , 64 , (1,1)) #for dimentionality control
        self.off2relu1 = nn.ReLU(inplace = True)
        
        self.off2pad = nn.ZeroPad2d(1)
        
        self.off2conv2 = nn.Conv2d(256 , 128 , (1,1) )
        self.off2relu2 = nn.ReLU(inplace = True)
        
        self.off2conv3 = nn.Conv2d(128 , 128 , (3,3) , padding= 1)
        self.off2relu3 = nn.ReLU(inplace = True)
        self.off2conv4 = nn.Conv2d(128 , 128 , (3,3) , padding= 1)
        self.off2relu4 = nn.ReLU(inplace = True)
        self.off2conv5 = nn.Conv2d(128 , 128 , (3,3) , padding= 1)
        self.off2relu5 = nn.ReLU(inplace = True)
        
        self.off2avgpool = nn.AvgPool2d((2,2))
        
        #OFF Sub-network unit three
        self.off3conv1 = nn.Conv2d(64 , 64 , (1,1)) #for dimentionality control
        self.off3relu1 = nn.ReLU(inplace = True)
        
        self.off3pad = nn.ZeroPad2d(1)
        
        self.off3conv2 = nn.Conv2d(320 , 128 , (1,1) )
        self.off3relu2 = nn.ReLU(inplace = True)
        self.off3conv3 = nn.Conv2d(128 , 128 , (3,3) , padding= 1)
        self.off3relu3 = nn.ReLU(inplace = True)
        self.off3conv4 = nn.Conv2d(128 , 128 , (3,3) , padding= 1)
        self.off3relu4 = nn.ReLU(inplace = True)
        self.off3conv5 = nn.Conv2d(128 , 128 , (3,3) , padding= 1)
        self.off3relu5 = nn.ReLU(inplace = True)
        
        self.off3avgpool = nn.AvgPool2d((2,2))
        
        self.offFinconv1 = nn.Conv2d(128 , 128 , (3,3) , padding= 1)
        self.offFinrelu1 = nn.ReLU(inplace = True)
        self.offFinconv2 = nn.Conv2d(128 , 128 , (3,3) , padding= 1)
        self.offFinrelu2 = nn.ReLU(inplace = True)
        
        self.offFinavgpool = nn.AvgPool2d((2,2))
        
        self.off_ffnn5 = nn.Linear(8192, 2048)
        self.off_relu5 = nn.ReLU(inplace = True)
        self.off_drop5 = nn.Dropout(p = 0.3)

        self.off_ffnn6 = nn.Linear(2048 , 101)
        self.off_softmax = nn.LogSoftmax(dim = 1)
        


    def forward (self , x) :

        # x shape should be like (batch_size*seq_size , 3 , 124 , 124)


        # Feature Generation Sub-network
        x1 = self.conv1a(x)
        x1 = self.relu1a(x1)
        x1 = self.conv1b(x1)
        x1 = self.relu1b(x1)

        x1_pooled = self.maxpool1(x1)

        x2 = self.conv2a(x1_pooled)
        x2 = self.relu2a(x2)
        x2 = self.conv2b(x2)
        x2 = self.relu2b(x2)

        x2_pooled = self.maxpool2(x2)

        x3 = self.conv3a(x2_pooled)
        x3 = self.relu3a(x3)
        x3 = self.conv3b(x3)
        x3 = self.relu3b(x3)
        x3 = self.conv3c(x3)
        x3 = self.relu3c(x3)

        x3_pooled = self.maxpool3(x3)

        x4 = self.conv4a(x3_pooled)
        x4 = self.relu4a(x4)
        x4 = self.conv4b(x4)
        x4 = self.relu4b(x4)

        x4 = self.avgpool4(x4)

        x4 = x4.view(x4.shape[0] , -1)

        x4 = self.ffnn5(x4)
        x4 = self.relu5(x4)
        x4 = self.drop5(x4)
        x4 = self.ffnn6(x4)
        x4 = self.softmax(x4)
        x4 = x4.view(self.batch_size , self.seq_size , 101)
        x4 = x4.mean(dim = 1)
        
        
        
        #-----------------------------------------------------------------------------------------------------
        
        #OFF Sub-network unit one
        
        x1 = self.off1conv1(x1)
        x1 = self.off1relu1(x1)
        
        
        #calculating the temporal gradient
        cur_channel_size = x1.shape[1]
        x1_bybatch = x1.view (self.batch_size , -1 , 126 , 126) #the shape is (batch_size , seq_len * channel_size , h,w)
        
        x1_last  = x1_bybatch[:, cur_channel_size : , : , :] #all the frames except the first one
        x1_first  = x1_bybatch [:, : -cur_channel_size, : , :] #all the frames except the last one
        
        x1T_reshaped = x1_last - x1_first #substracting the consective frames from each other
        x1T = x1T_reshaped.view(-1 , cur_channel_size , 126 , 126) #the shape is (batch_size* (seq_len-1) , channel_size , h,w)
        
        
        #computing the spatial gradient
        x1_spatial = x1[: self.batch_size * (self.seq_size -1), :, : , :] #discarding the last frame
        x1XY = sobelOps(x1_spatial) #the shape is (batch_size* (seq_len-1) , channel_size , h,w)
        
        #stacking the gradients
        x1XYT = torch.cat([x1XY , x1T] , dim = 1) #shape (batch_size* (seq_len-1) , 3*channel_size , h,w)
        
        x1_off1_red = self.off1conv2(x1XYT)
        x1_off1_red = self.off1relu2(x1_off1_red)
        
        x1_off1_out = self.off1conv3(x1_off1_red)
        x1_off1_out = self.off1relu3(x1_off1_out)
        x1_off1_out = self.off1conv4(x1_off1_out)
        x1_off1_out = self.off1relu4(x1_off1_out)
        x1_off1_out = self.off1conv5(x1_off1_out)
        x1_off1_out = self.off1relu5(x1_off1_out)
        
        x1_off1_output = x1_off1_out + x1_off1_red
        
        
        x1_off1_output_pooled = self.off1avgpool(x1_off1_output)
        
        
        #-----------------------------------------------------------------------------------------------------
        
        #OFF Sub-network unit two
        
        x2 = self.off2conv1(x2)
        x2 = self.off2relu1(x2)
        
        #calculating the temporal gradient
        cur_channel_size = x2.shape[1]
        x2_bybatch = x2.view (self.batch_size , -1 , 62 , 62) #the shape is (batch_size , seq_len * channel_size , h,w)
        
        x2_last  = x2_bybatch[:, cur_channel_size : , : , :] #all the frames except the first one
        x2_first  = x2_bybatch [:, : -cur_channel_size, : , :] #all the frames except the last one
        
        x2T_reshaped = x2_last - x2_first #substracting the consective frames from each other
        x2T = x2T_reshaped.view(-1 , cur_channel_size , 62 , 62) #the shape is (batch_size* (seq_len-1) , channel_size , h,w)
        
        
        #computing the spatial gradient
        x2_spatial = x2[: self.batch_size * (self.seq_size -1), :, : , :] #discarding the last frame
        x2XY = sobelOps(x2_spatial) #the shape is (batch_size* (seq_len-1) , channel_size , h,w)
        
        #stacking the gradients
        x2XYT = torch.cat([x2XY , x2T ] , dim = 1) #shape (batch_size* (seq_len-1) , 3*channel_size , h,w)
        x2XYT = self.off2pad(x2XYT)
        
        
        x2XYT = torch.cat([x2XYT , x1_off1_output_pooled ] , dim = 1) #shape (batch_size* (seq_len-1) , 3*channel_size + prev_chan_size , h,w)
        
        x2_off2_red = self.off2conv2(x2XYT)
        x2_off2_red = self.off2relu2(x2_off2_red)
        
        x2_off2_out = self.off2conv3(x2_off2_red)
        x2_off2_out = self.off2relu3(x2_off2_out)
        x2_off2_out = self.off2conv4(x2_off2_out)
        x2_off2_out = self.off2relu4(x2_off2_out)
        x2_off2_out = self.off2conv5(x2_off2_out)
        x2_off2_out = self.off2relu5(x2_off2_out)
        
        
        x2_off2_output = x2_off2_out + x2_off2_red
        
        x2_off2_output_pooled = self.off2avgpool(x2_off2_output)
        
        #-----------------------------------------------------------------------------------------------------
        
        
        #OFF Sub-network unit three
        
        x3 = self.off3conv1(x3)
        x3 = self.off3relu1(x3)
        
        #calculating the temporal gradient
        cur_channel_size = x3.shape[1]
        x3_bybatch = x3.view (self.batch_size , -1 , 30 , 30) #the shape is (batch_size , seq_len * channel_size , h,w)
        
        x3_last  = x3_bybatch[:, cur_channel_size : , : , :] #all the frames except the first one
        x3_first  = x3_bybatch [:, : -cur_channel_size, : , :] #all the frames except the last one
        
        x3T_reshaped = x3_last - x3_first #substracting the consective frames from each other
        x3T = x3T_reshaped.view(-1 , cur_channel_size , 30 , 30) #the shape is (batch_size* (seq_len-1) , channel_size , h,w)
        
        
        #computing the spatial gradient
        x3_spatial = x3[: self.batch_size * (self.seq_size -1), :, : , :] #discarding the last frame
        x3XY = sobelOps(x3_spatial) #the shape is (batch_size* (seq_len-1) , channel_size , h,w)
        
        #stacking the gradients
        x3XYT = torch.cat([x3XY , x3T] , dim = 1) #shape (batch_size* (seq_len-1) , 3*channel_size , h,w)
        x3XYT = self.off3pad(x3XYT)
        
        x3XYT = torch.cat([x3XYT , x2_off2_output_pooled ] , dim = 1) #shape (batch_size* (seq_len-1) , 3*channel_size + prev_chan_size , h,w)
        
        x3_off3_red = self.off3conv2(x3XYT)
        x3_off3_red = self.off3relu2(x3_off3_red)
        
        x3_off3_out = self.off3conv3(x3_off3_red)
        x3_off3_out = self.off3relu3(x3_off3_out)
        x3_off3_out = self.off3conv4(x3_off3_out)
        x3_off3_out = self.off3relu4(x3_off3_out)
        x3_off3_out = self.off3conv5(x3_off3_out)
        x3_off3_out = self.off3relu5(x3_off3_out)
        
        x3_off3_output = x3_off3_out + x3_off3_red
        
        x3_off3_output_pooled = self.off3avgpool(x3_off3_output)
        
        final_off_output = self.offFinconv1(x3_off3_output_pooled)
        final_off_output = self.offFinrelu1(final_off_output)
        final_off_output = self.offFinconv2(final_off_output)
        final_off_output = self.offFinrelu2(final_off_output)
        
        final_off_output = self.offFinavgpool(final_off_output)
        
        final_off_output = final_off_output.view(final_off_output.shape[0] , -1)
        
        final_off_output = self.off_ffnn5(final_off_output)
        final_off_output = self.off_relu5(final_off_output)
        final_off_output = self.off_drop5(final_off_output)
        final_off_output = self.off_ffnn6(final_off_output)
        final_off_output = self.off_softmax(final_off_output)
        
        final_off_output = final_off_output.view(self.batch_size , self.seq_size - 1  , 101)
        final_off_output = final_off_output.mean(dim = 1)
        
        output = (final_off_output + x4)/2
        
        return output
            

In [4]:
glob.glob("UCF-101/*")[94:95]

['UCF-101\\Typing']

In [6]:
folders = glob.glob("UCF-101/*")

for Vfolder in folders[94:95]:
    files = glob.glob(Vfolder + "/*.avi")
    print(files)

['UCF-101\\Typing\\v_Typing_g01_c01.avi', 'UCF-101\\Typing\\v_Typing_g01_c02.avi', 'UCF-101\\Typing\\v_Typing_g01_c03.avi', 'UCF-101\\Typing\\v_Typing_g01_c04.avi', 'UCF-101\\Typing\\v_Typing_g01_c05.avi', 'UCF-101\\Typing\\v_Typing_g01_c06.avi', 'UCF-101\\Typing\\v_Typing_g01_c07.avi', 'UCF-101\\Typing\\v_Typing_g02_c01.avi', 'UCF-101\\Typing\\v_Typing_g02_c02.avi', 'UCF-101\\Typing\\v_Typing_g02_c03.avi', 'UCF-101\\Typing\\v_Typing_g02_c04.avi', 'UCF-101\\Typing\\v_Typing_g02_c05.avi', 'UCF-101\\Typing\\v_Typing_g02_c06.avi', 'UCF-101\\Typing\\v_Typing_g03_c01.avi', 'UCF-101\\Typing\\v_Typing_g03_c02.avi', 'UCF-101\\Typing\\v_Typing_g03_c03.avi', 'UCF-101\\Typing\\v_Typing_g03_c04.avi', 'UCF-101\\Typing\\v_Typing_g03_c05.avi', 'UCF-101\\Typing\\v_Typing_g03_c06.avi', 'UCF-101\\Typing\\v_Typing_g03_c07.avi', 'UCF-101\\Typing\\v_Typing_g04_c01.avi', 'UCF-101\\Typing\\v_Typing_g04_c02.avi', 'UCF-101\\Typing\\v_Typing_g04_c03.avi', 'UCF-101\\Typing\\v_Typing_g04_c04.avi', 'UCF-101\\Typin

In [8]:
files[141:]

[]

In [16]:
folders[94:95]

['UCF-101\\Typing']

In [20]:
folders = glob.glob("UCF-101/*")

for Vfolder in folders[94:95]:
    
    foldername = Vfolder[8:] #You keep only the name of the last folder
    
    files = glob.glob(Vfolder + "/*.avi")
    
    
    for F in files[135:]:
        print(F)

UCF-101\Typing\v_Typing_g25_c04.avi


In [31]:
import os
import sys
import subprocess
import glob
import time


start = time.time()

folders = glob.glob("UCF-101/*")

for Vfolder in folders[94:95]:
    
    
    foldername = Vfolder[8:] #You keep only the name of the last folder
    
    files = glob.glob(Vfolder + "/*.avi")
    
    elapsed_time = time.time() - start
    
    
    
    print (f"elapsed_time_{foldername} : {elapsed_time} [sec]")
    
    for F in files[135:]:
        
        name = F.split("\\")
        
        filename = name[2]
        filename = filename[:-4]
        subprocess.getoutput("mkdir TrajectorySet/TrajectorySet/media/gwladys/1DD3E28B3E6EB2D5/result/%s " %(foldername))
        subprocess.getoutput("TrajectorySet/TrajectorySet/release/DenseTrackStab %s /media/gwladys/1DD3E28B3E6EB2D5/result/%s/%s.txt \
        " %(F,foldername,filename))

elapsed_time_Typing : 0.004001140594482422 [sec]


In [32]:
from distutils.core import setup
from distutils.extension import Extension
from Cython.Build import cythonize

In [38]:
from distutils.core import setup
from distutils.extension import Extension
from Cython.Build import cythonize

ext = [
    Extension(
        "fisher",
        ["TrajectorySet/TrajectorySet/python/fisher.pyx"],
        extra_compile_args=['-fopenmp'],
        extra_link_args=['-fopenmp']
    )
]

setup(
    name='fisher',
    ext_modules = cythonize(ext)
)

Compiling TrajectorySet/TrajectorySet/python/fisher.pyx because it changed.
[1/1] Cythonizing TrajectorySet/TrajectorySet/python/fisher.pyx


C:\ProgramData\Anaconda3\lib\site-packages\Cython\Compiler\Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\Mohie Eldeen Elzeiny\Desktop\Action recognition\TrajectorySet\TrajectorySet\python\fisher.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


SystemExit: usage: ipykernel_launcher.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: ipykernel_launcher.py --help [cmd1 cmd2 ...]
   or: ipykernel_launcher.py --help-commands
   or: ipykernel_launcher.py cmd --help

error: option -f not recognized

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [40]:
!python "C:/Users/Mohie Eldeen Elzeiny/Desktop/Action recognition/TrajectorySet/TrajectorySet/python/setup.py" build_ext --inplace

Traceback (most recent call last):
  File "C:/Users/Mohie Eldeen Elzeiny/Desktop/Action recognition/TrajectorySet/TrajectorySet/python/setup.py", line 16, in <module>
    ext_modules = cythonize(ext)
  File "C:\ProgramData\Anaconda3\lib\site-packages\Cython\Build\Dependencies.py", line 967, in cythonize
    aliases=aliases)
  File "C:\ProgramData\Anaconda3\lib\site-packages\Cython\Build\Dependencies.py", line 812, in create_extension_list
    for file in nonempty(sorted(extended_iglob(filepattern)), "'%s' doesn't match any files" % filepattern):
  File "C:\ProgramData\Anaconda3\lib\site-packages\Cython\Build\Dependencies.py", line 111, in nonempty
    raise ValueError(error_msg)
ValueError: 'fisher.pyx' doesn't match any files
